In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import sklearn
sklearn.set_config(display='diagram') #П

# from tqdm.auto import tqdm
# tqdm.pandas()
import nltk
from nltk.tokenize import sent_tokenize

from sklearn.ensemble import RandomForestRegressor, VotingRegressor
from sklearn.decomposition import TruncatedSVD, PCA
from scipy.stats import rankdata


import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import Ridge,RidgeCV, LinearRegression, Lasso
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import StandardScaler
from mlxtend.feature_selection import ColumnSelector
from sklearn.compose import ColumnTransformer, make_column_selector

from nltk.corpus import stopwords
from pprint import pprint
from time import time
import logging

from bs4 import BeautifulSoup
import re
import unidecode

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

# Display progress logs on stdout
logging.basicConfig(level=logging.INFO, format="%(asctime)s %(levelname)s %(message)s")



In [ ]:
!pip install --no-index --no-deps ../input/swifter/swifter-1.1.2.tar

# Settings

In [ ]:
# !pip install --no-index --no-deps   ../input/pyphen-0100/*.whl
# !pip install --no-index --no-deps   ../input/textstat071py3/*.whl

In [ ]:
# import textstat

# Custom Stop Words

In [ ]:
# stop = stopwords.words('english')

# not_stop_word = ['i','me','my','myself','we','our','ours','ourselves','you',"you're","you've","you'll","you'd",'your','yours','yourself','yourselves',
#                  'he','him','his','himself','she',"she's",'her', 'hers','herself','it',"it's",'its','itself','they','them','their','theirs','themselves',
#                  'what','are','at','by','for','with','about','against', 'between','into','through', 'to','from','up','down','in','out','on','off','over',
#                  'under','again','then','once','here','there','when', 'why','how','all','any', 'each','more','most','other','such','no','not',
#                  'only','own','same','so','than','too','very','will','just', "don't",'should',"should've",'now','d','ll','m','o','re', 'ain','aren',"aren't",'couldn',"couldn't",]

# filtered_stop_word = [i for i in stop if i not in not_stop_word]

In [ ]:
def clean(data, col):
    
    def remove_urls (vTEXT):
        vTEXT = re.sub(r'http\S+|www\S+', 'URL', vTEXT)
        return(vTEXT)

    data[col] = data[col].swifter.apply(remove_urls)
    
    data[col] = data[col].str.replace(r"-", " - ", regex=True)
    
    #Removing numbers
    data[col] = data[col].swifter.apply(lambda x : ' '.join([tweet for tweet in x.split() if not tweet == '\d*']))
    
        # Remove ip address
    data[col] = data[col].str.replace(r'(([0-9]+\.){2,}[0-9]+)',' ', regex=True)
    # Remove website
    data[col] = data[col].str.replace(r'https?://\S+|www\.\S+', ' ', regex=True)
    data[col] = data[col].str.replace(r'([a-zA-Z]+)([/!?.])([a-zA-Z]+)',r'\1 \2 \3', regex=True)
    # Replace repeating characters more than 3 times to length of 3
    data[col] = data[col].str.replace(r'([*!?\'])\1\1{2,}',r'\1\1\1', regex=True)
    # patterns with repeating characters 
    data[col] = data[col].str.replace(r'([a-zA-Z])\1{2,}\b',r'\1\1', regex=True)
    data[col] = data[col].str.replace(r'([a-zA-Z])\1\1{2,}\B',r'\1\1\1', regex=True)
    data[col] = data[col].str.replace(r'[ ]{2,}',' ', regex=True).str.strip()   
    # Add space around repeating characters
    data[col] = data[col].str.replace(r'([*!?\'"]+)',r' \1 ', regex=True)    
    # Remove multiple white spaces
    data[col] = data[col].str.replace(r' +', ' ', regex=True)
    # Remove html tags
    data[col] = data[col].str.replace(r'<[^<]+?>', ' ', regex=True)
    
    #Code for removing slang words
    
    d = {'luv':'love','wud':'would','lyk':'like','wateva':'whatever','ttyl':'talk to you later',
         'kul':'cool','fyn':'fine','omg':'oh my god!','fam':'family','bruh':'brother',
         'cud':'could', 'fud':'food'} ## Need a huge dictionary
        
    data[col] = data[col].swifter.apply(lambda x : ' '.join(d[word] if word in d else word for word in x.split()))
    data[col] = data[col].str.replace(r"j k", "jk", regex=True)
    data[col] = data[col].str.replace(r"e - mail", "email", regex=True)
    data[col] = data[col].str.replace(r"\0s", "0", regex=True)
    data[col] = data[col].str.replace(r" 9 11 ", " 911 ", regex=True)
    data[col] = data[col].str.replace(r" e g ", " eg ", regex=True)
    data[col] = data[col].str.replace(r" b g ", " bg ", regex=True)
    data[col] = data[col].str.replace(r" u s ", " american ", regex=True)
    data[col] = data[col].str.replace(r"what's", "what is ", regex=True)
    data[col] = data[col].str.replace(r"\'ve", " have ", regex=True)
    data[col] = data[col].str.replace(r"can't", "cannot ", regex=True)
    data[col] = data[col].str.replace(r"n't", " not ", regex=True)
    data[col] = data[col].str.replace(r"i'm", "i am ", regex=True)
    data[col] = data[col].str.replace(r"\'re", " are ", regex=True)
    data[col] = data[col].str.replace(r"\'d", " would ", regex=True)
    data[col] = data[col].str.replace(r"\'ll", " will ", regex=True)
    data[col] = data[col].str.replace(r"\'scuse", " excuse ", regex=True)
    data[col] = data[col].str.replace(r"\'s", " ", regex=True)
    data[col] = data[col].str.replace('\n', ' \n ', regex=True)
    data[col] = data[col].str.replace(r'([a-zA-Z]+)([/!?.])([a-zA-Z]+)', r'\1 \2 \3', regex=True)
    data[col] = data[col].str.replace(r'([*!?\'])\1\1{2,}', r'\1\1\1', regex=True)
    data[col] = data[col].str.replace(r'([*!?\']+)', r' \1 ', regex=True)
    data[col] = data[col].str.replace(r'([a-zA-Z])\1{2,}\b', r'\1\1', regex=True)
    data[col] = data[col].str.replace(r'([a-zA-Z])\1\1{2,}\B', r'\1\1\1', regex=True)
    data[col] = data[col].str.replace(r'[ ]{2,}', ' ', regex=True).str.strip()
    data[col] = data[col].str.replace(r'[ ]{2,}', ' ', regex=True).str.strip()

    def count_urls(text):
        return text.count('URL')
    
    data['URL_count'] = data[col].swifter.apply(count_urls)

#     data[col] = data[col].swifter.apply(lambda x: ' '.join([word for word in x.split() if word not in (filtered_stop_word)]))
    return data

## Feature Engineering

In [ ]:
# pd.read_csv('../input/jigsaw-toxic-severity-rating/comments_to_score.csv')
f = pd.read_csv('../input/jigsaw-toxic-severity-rating/validation_data.csv')
f.sample(10)

In [ ]:
f.sample(3)

In [ ]:
pd.options.display.max_colwidth = 500
pd.options.display.max_rows = 50
f[['more_toxic']].sample(50)

In [ ]:
import swifter

# Feature Engineering

In [ ]:
from string import punctuation

def up_low(s):
    u, l  = sum(1 for i in s if i.isupper()), sum(1 for i in s if i.islower())
    if l == 0 : l = 1
    return u/l

def up_low_word(s):
    u, l  = sum(1 for i in s.split() if i.isupper()), sum(1 for i in s.split() if i.islower())
    if l == 0 : l = 1
    return u/l


# расстояние левинштейна


def get_toxic_index(text, words = ['fuck', 'bitch', None]):
    sen_list = sent_tokenize(text)
    sen_list = [sen.lower() for sen in sen_list]
    one_word_sum = 0
    sent_words = len(text.split())
    text = text.lower()
    
    
    for word in words:
        one_word_sum += text.count(word)/sent_words*np.log(1 + (len(sen_list)+1)/(sum(1 for sen in sen_list if sen.count(word))+1))
    return one_word_sum


def make_features(df, target = 'text', show_fs=False):
    
    col_before = df.columns
    
    df['sentence_count'] = df[target].swifter.apply(lambda x : len(sent_tokenize(x)))
    df['upper_on_lower'] = df[target].swifter.apply(up_low)
    df['upper_on_lower_word'] = df[target].swifter.apply(up_low_word)
    df['len_text'] = df[target].swifter.apply(lambda x : len(x)//30)
    df['size_punct'] = df[target].swifter.apply(lambda x : sum(1 for i in x if i in punctuation)) # Надо сколько именно слов, не символов
    df['size_letter'] = df[target].swifter.apply(lambda x : sum(1 for i in x if i not in punctuation))
    df['punct_on_letter'] = df['size_punct'].values/df['size_letter'].values
    df.drop(['size_letter', 'size_punct'], axis=1, inplace=True)
#     df['mean_word_len'] = df[target].swifter.apply(lambda x : sum(len(i)//3 for i in x.split())/len(x.split()))
    df['words'] = df[target].swifter.apply(lambda x : sum(1 for i in x.split())/len(x))
    df['ques_punct_on_word'] = df[target].swifter.apply(lambda x : 100*sum(1 for i in x if i in '?')/len(x))
    df['state_punct_on_word'] = df[target].swifter.apply(lambda x : 100*sum(1 for i in x if i in '!')/len(x))
    df['duble_state_punct_on_word'] = df[target].swifter.apply(lambda x : 100*sum(1 for i in x if i in '!!')/len(x))
    df['snow_punct_on_word'] = df[target].swifter.apply(lambda x : 100*sum(1 for i in x if i in '*')/len(x))
    df['uniq_word_on_sentence'] = df[target].swifter.apply(lambda x : len(set(x.split()))/len(x.split()))*df['sentence_count'].values
    df['haha_pattern'] = df[target].swifter.apply(lambda x : x.lower().count('hah')/len(x.split()))*df['sentence_count'].values
    df['fuck_pattern'] = df[target].swifter.apply(lambda x : x.lower().count('fuck')/len(x.split()))*df['sentence_count'].values
    df['off_pattern'] = df[target].swifter.apply(lambda x : x.lower().count('off')/len(x.split()))*df['sentence_count'].values
    df['ques_pattern'] = df[target].swifter.apply(lambda x : x.count('???')/len(x.split()))*df['sentence_count'].values
    df['you_pattern'] = df[target].swifter.apply(lambda x : x.lower().count(' you')/len(x.split()))*df['sentence_count'].values

    ###############################
    
    you_are_patterns = ['you are', 'you ar ', 'you re ', 'u are ', ' u r ', ' u ', ' are you', ' ur ', "you ' re", "you're", 'youre',
                        'you will', "you'll", "you ' ll", 'u will'] 
    
    print('Первый признак через нормировку пошел')
#     df['you_are_patterns'] = df[target].swifter.apply(lambda x : sum(len(x.split(i)) for i in you_are_patterns )/len(x.split()))*df['sentence_count'].values
    df['you_are_patterns'] = df[target].swifter.apply(get_toxic_index, words = you_are_patterns)
    print('Первый признак через нормировку готов')
    
    hide_punct_pattern = '*&%$#@'
#     df['hide_punct_pattern'] = df[target].swifter.apply(lambda x : sum(x.count(i) for i in hide_punct_pattern )/len(x.split()))*df['sentence_count'].values
    df['hide_punct_pattern'] = df[target].swifter.apply(get_toxic_index, words = hide_punct_pattern)

    penis_pattern = ['dick','diick', 'penis', 'sock','suck', 'cock', 'balls', ' ball', 'bell', 'phallus','condone',
                     'sucker', 'suker', 'penes', 'prick', 'pecker', 'dildo']
#     df['penis_pattern'] = df[target].swifter.apply(lambda x : sum(x.lower().count(word) for word in penis_pattern)/len(x.split()))*df['sentence_count'].values
    df['penis_pattern'] = df[target].swifter.apply(get_toxic_index, words = penis_pattern)
    
    fucking_pattern = ['fuck', '*uck', 'f*ck', 'fu*k', 'fuc*','f**k','f ** k', 'fucck','fcuken','fcken','ruck', 'f *** ing', 'sh * t', 'sh*t','s**t', 'fukk', 'fricki',
                       'f***', 'f ** ck', 'f * ck', 'b ** ch','b * ch']
#     df['fucking_pattern'] = df[target].swifter.apply(lambda x : sum(x.lower().count(word) for word in fucking_pattern)/len(x.split()))*df['sentence_count'].values  
    df['fucking_pattern'] = df[target].swifter.apply(get_toxic_index, words = fucking_pattern)
    
    virgin_pattern = ['virgin', 'slut', 'whore', 'prostitut', 'bitch', 'biatch', 'hooker','pussy', 'beetch', 'bithc', 'butch']
#     df['virgin_pattern'] = df[target].swifter.apply(lambda x : sum(x.lower().count(word) for word in virgin_pattern)/len(x.split()))*df['sentence_count'].values 
    df['virgin_pattern'] = df[target].swifter.apply(get_toxic_index, words = virgin_pattern)
    
    dubble_word_pattern = ['buttseck', 'dumbass', 'asshole', 'arsehole', 'cocksucker', 'asscake', 'jackass']
#     df['dubble_word_pattern'] = df[target].swifter.apply(lambda x : sum(x.lower().count(word) for word in dubble_word_pattern)/len(x.split()))*df['sentence_count'].values
    df['dubble_word_pattern'] = df[target].swifter.apply(get_toxic_index, words = dubble_word_pattern)
    
    funny_pattern = ['glad', 'happy', 'hilarious', 'fanny', 'humor']
#     df['funny_pattern'] = df[target].swifter.apply(lambda x : sum(x.lower().count(word) for word in funny_pattern)/len(x.split()))*df['sentence_count'].values
    df['funny_pattern'] = df[target].swifter.apply(get_toxic_index, words = funny_pattern)
    
   # ДОбавить поправку растояния левенштена                                                                       
    toxic_words = ['ass', 'a$$', 'hole', 'drool', 'anal', 'pedo', 'poopie', 'hose', 'berk', 'duffer', 'pillock', 'plonker', 'wally',
                   'knobend', 'rotter', 'swine', 'blighter', ' cad', 'idiot', 'faggot', 'clown', 'tosser', 'cunt', 'arrogant', 'freak', 'dude',  
                   'ugly', 'little', 'fat', 'liar', 'heck', 'vandal', 'nerd','stubborn']
#     df['toxic_word_pattern'] = df[target].swifter.apply(lambda x : sum(x.lower().count(word) for word in toxic_words)/len(x.split()))*df['sentence_count'].values 
    df['toxic_word_pattern'] = df[target].swifter.apply(get_toxic_index, words = toxic_words)    
        
    yeald_pattern = ['ggg','uuu','aaa', 'hhh', 'eee','ooo','!!!','???', '!?!', '!??']
#     df['yeald_pattern'] = df[target].swifter.apply(lambda x : sum(x.lower().count(word) for word in yeald_pattern)/len(x.split()))*df['sentence_count'].values    
    df['yeald_pattern'] = df[target].swifter.apply(get_toxic_index, words = yeald_pattern)    
        
    noncence_toxic = ['poo', 'poop', 'poopy', 'butt','piss','shit','shut', 'testicles', 'bollocks', 'heresy', 'absurd', 'nonsense', 'crud','crap']
#     df['noncence_toxic'] = df[target].swifter.apply(lambda x : sum(x.lower().count(word) for word in noncence_toxic)/len(x.split()))*df['sentence_count'].values
    df['noncence_toxic'] = df[target].swifter.apply(get_toxic_index, words = noncence_toxic)
    
    long_phase_pattern = ['fuck all', 'piss off', 'fuck off', 'get off', 'stay off', 'shut up', 'god damn', 'damn it', 'who care', 'kiss off', 'fuck–up', 'fuck up', 'bang up']
#     df['long_phase_pattern'] = df[target].swifter.apply(lambda x : sum(x.lower().count(word) for word in long_phase_pattern)/len(x.split()))*df['sentence_count'].values
    df['long_phase_pattern'] = df[target].swifter.apply(get_toxic_index, words = long_phase_pattern)

    child_toxic = ['holy', 'moly', 'jeepers', 'good', 'heavens', 'gosh', 'fiddlesticks', 'poppycock']
#     df['child_toxic'] = df[target].swifter.apply(lambda x : sum(x.lower().count(word) for word in child_toxic)/len(x.split()))*df['sentence_count'].values
    df['child_toxic'] = df[target].swifter.apply(get_toxic_index, words = child_toxic)

    more_toxic_word = ['damn', 'whore', 'hoe', 'jade', 'jerk', 'moron', 
                       'douchebag', 'dork', 'faggot', 'fag', 'queer', 'pirate', 'candyass',
                       'finook', 'capon', 'butter', 'boy', 'banana', 'crammer', 'nancy', 'pansy', 'scumbag', 'scum', 'prat', 'brat', 'loser',
                       'sack', 'goof', 'wanker', 'noob', 'retard', 'numbnuts', 'shitbox']
#     df['more_toxic_word'] = df[target].swifter.apply(lambda x : sum(x.lower().count(word) for word in more_toxic_word)/len(x.split()))*df['sentence_count'].values
    df['more_toxic_word'] = df[target].swifter.apply(get_toxic_index, words = more_toxic_word)
    
    light_level_toxic  = ['arse', 'bloody', 'hell', 'bugger', 'damn', 'minger', 'sod-off']
#     df['light_level_toxic'] = df[target].swifter.apply(lambda x : sum(x.lower().count(word) for word in light_level_toxic)/len(x.split()))*df['sentence_count'].values
    df['light_level_toxic'] = df[target].swifter.apply(get_toxic_index, words = light_level_toxic)
    
    middle_level_toxic = ['shit', 'son of a bitch', 'arsehole', 'balls', 'bint', 'bollocks', 'bullshit', 'feck', 'munter', 'pissed', 'pissed off']
#     df['middle_level_toxic'] = df[target].swifter.apply(lambda x : sum(x.lower().count(word) for word in middle_level_toxic)/len(x.split()))*df['sentence_count'].values
    df['middle_level_toxic'] = df[target].swifter.apply(get_toxic_index, words = middle_level_toxic)
    
    hard_level_toxic  = ['bastard', 'dickhead', 'bloodclaat', 'jamaican', 'knob', 'prick', 'bellend',
                        'minge', 'twat', 'twunt', 'beaver', 'curtains', 'beef', 'clunge', 'snatch', 'punani', 'gash']
#     df['hard_level_toxic'] = df[target].swifter.apply(lambda x : sum(x.lower().count(word) for word in hard_level_toxic)/len(x.split()))*df['sentence_count'].values
    df['hard_level_toxic'] = df[target].swifter.apply(get_toxic_index, words = hard_level_toxic)
    
    very_hard_level_toxic  = ['cunt', 'fuck', 'motherfucker', 'motherfudger', 'mother fudger']
#     df['very_hard_level_toxic'] = df[target].swifter.apply(lambda x : sum(x.lower().count(word) for word in very_hard_level_toxic)/len(x.split()))*df['sentence_count'].values
    df['very_hard_level_toxic'] = df[target].swifter.apply(get_toxic_index, words = very_hard_level_toxic)
    
    very_very_hard_level_toxic  = ['felch' ,'skullfuck', 'fuck puppet', 'bisnotch', 'rusty trombone']
#     df['very_very_hard_level_toxic'] = df[target].swifter.apply(lambda x : sum(x.lower().count(word) for word in very_very_hard_level_toxic)/len(x.split()))*df['sentence_count'].values
    df['very_very_hard_level_toxic'] = df[target].swifter.apply(get_toxic_index, words = very_very_hard_level_toxic)
    
    away_patterns  = [' out ', 'away ', ' off ' ,' up ', ' down ', 'm done']
#     df['away_pattern'] = df[target].swifter.apply(lambda x : sum(x.lower().count(word) for word in away_patterns)/len(x.split()))*df['sentence_count'].values
    df['away_patterns'] = df[target].swifter.apply(get_toxic_index, words = away_patterns) 
        
    synon_toxic_words = ['jerk', 'stinker', 'heck', 'crud', 'schmuck', 'riffraff']    
#     df['synon_toxic_words'] = df[target].swifter.apply(lambda x : sum(x.lower().count(word) for word in synon_toxic_words)/len(x.split()))*df['sentence_count'].values
    df['synon_toxic_words'] = df[target].swifter.apply(get_toxic_index, words = synon_toxic_words)
    
    lgbt_pattern = [ 'gay', 'homosexual', 'hetero', ' lgbt', 'bisexual', 'lesbian', 'queer']
#     df['lgbt_pattern'] = df[target].swifter.apply(lambda x : sum(x.lower().count(word) for word in lgbt_pattern)/len(x.split()))*df['sentence_count'].values
    df['lgbt_pattern'] = df[target].swifter.apply(get_toxic_index, words = lgbt_pattern)
    
    descrimination_pattern = ['racist', 'nazi', 'antic', 'commie',  'discriminat', 'female', 'troll', 'abusing', 'nigger', 'nigga', 'jew',
                            'fascist', 'haras', 'racism', 'bigot', 'gosh', 'kike', 'slave', 'victim', ]
#     df['descrimination_pattern'] = df[target].swifter.apply(lambda x : sum(x.lower().count(word) for word in descrimination_pattern)/len(x.split()))*df['sentence_count'].values
    df['descrimination_pattern'] = df[target].swifter.apply(get_toxic_index, words = descrimination_pattern)
    
    god_pattern = ['catholic', 'god','religi', 'islam', 'bibl', 'muslim', 'atheist']
#     df['god_pattern'] = df[target].swifter.apply(lambda x : sum(x.lower().count(word) for word in god_pattern)/len(x.split()))*df['sentence_count'].values
    df['god_pattern'] = df[target].swifter.apply(get_toxic_index, words = god_pattern)
    
    animal_pattern = ['monkey', 'horse', 'pig', 'animal', 'dog', 'bull', ' swine', ' pork', 'llama','skunk','racooon', 'raccoon', 'racoon', ' coon',
                    'bat', 'bird', 'chicken', 'duck', 'goose', 'weasel', ' swan', 'dog', 'donkey', 'goat', ' rat', 'sheep']
#     df['animal_pattern'] = df[target].swifter.apply(lambda x : sum(x.lower().count(word) for word in animal_pattern)/len(x.split()))*df['sentence_count'].values
    df['animal_pattern'] = df[target].swifter.apply(get_toxic_index, words = animal_pattern)
    
    slang_pattern = [' u ', ' ur ', 'cuz']
#     df['slang_pattern'] = df[target].swifter.apply(lambda x : sum(x.lower().count(word) for word in slang_pattern)/len(x.split()))*df['sentence_count'].values
    df['slang_pattern'] = df[target].swifter.apply(get_toxic_index, words = slang_pattern)
    
    body_pattern = ['tits', 'mouth', 'body', 'face',  'head', ' boob', 'oral']  
#     df['body_pattern'] = df[target].swifter.apply(lambda x : sum(x.lower().count(word) for word in body_pattern)/len(x.split()))*df['sentence_count'].values
    df['body_pattern'] = df[target].swifter.apply(get_toxic_index, words = body_pattern)
    
    not_good_words = ['rude', 'stuck', 'porn', 'mug', 'sex', 'shame', 'gang', 'bang', 'wanker']  
#     df['not_good_words'] = df[target].swifter.apply(lambda x : sum(x.lower().count(word) for word in not_good_words)/len(x.split()))*df['sentence_count'].values
    df['not_good_words'] = df[target].swifter.apply(get_toxic_index, words = not_good_words)
    
    threat_pattern = ['going', 'wanna', 'will', 'threat', 'barbarian', 'savage', 'insult', 'rape',]
#     df['threat_pattern'] = df[target].swifter.apply(lambda x : sum(x.lower().count(word) for word in threat_pattern)/len(x.split()))*df['sentence_count'].values
    df['threat_pattern'] = df[target].swifter.apply(get_toxic_index, words = threat_pattern)
    
    print('Примерно половина признаков готова!')
    hard_threat_words = ['xenocidic', 'terrorist', 'akbar', 'dangerous', 'genocide', 'holocaust']
#     df['hard_threat_words'] = df[target].swifter.apply(lambda x : sum(x.lower().count(word) for word in hard_threat_words)/len(x.split()))*df['sentence_count'].values
    df['hard_threat_words'] = df[target].swifter.apply(get_toxic_index, words = hard_threat_words)
    
    stop_pattern = ['delet', 'destroy', ' bann', 'block',  'stop', 'quit', 'unblock', 'attack', 'remove']
#     df['stop_pattern'] = df[target].swifter.apply(lambda x : sum(x.lower().count(word) for word in stop_pattern)/len(x.split()))*df['sentence_count'].values
    df['stop_pattern'] = df[target].swifter.apply(get_toxic_index, words = stop_pattern)
    
    mom_pattern = [' mom ', 'mother', ' moth ', ' mum ', 'mothefucker', ' milf', ]  
#     df['mom_pattern'] = df[target].swifter.apply(lambda x : sum(x.lower().count(word) for word in mom_pattern)/len(x.split()))*df['sentence_count'].values
    df['mom_pattern'] = df[target].swifter.apply(get_toxic_index, words = mom_pattern)
    
    brother_pattern = [' son ', 'son of', 'children', 'cousin', 'kids', 'baby', 'brother', 'father','family', 'daugher', 'sister']  
#     df['brother_pattern'] = df[target].swifter.apply(lambda x : sum(x.lower().count(word) for word in brother_pattern)/len(x.split()))*df['sentence_count'].values
    df['brother_pattern'] = df[target].swifter.apply(get_toxic_index, words = brother_pattern)
    
    mental_pattern = ['retard', 'mind', 'mental', 'brain', 'stupid', 'silly', 'fool', 'dumb', 'meatball', 'numnut',
                      'paranoid', 'mad', 'crazy', 'insane', 'frantic', 'reckless', 'lunatic', 'sick']  
#     df['mental_pattern'] = df[target].swifter.apply(lambda x : sum(x.lower().count(word) for word in mental_pattern)/len(x.split()))*df['sentence_count'].values
    df['mental_pattern'] = df[target].swifter.apply(get_toxic_index, words = mental_pattern)
    
    negative_pattern = ['horrible', 'coward', 'crack','less','dart','hate', 'annoy', 'hurt', 'demon',
                        'pain', 'dead', ' die',' death', 'satan', 'terrible', 'awful', 'spam', 'avil']
#     df['negative_pattern'] = df[target].swifter.apply(lambda x : sum(x.lower().count(word) for word in negative_pattern)/len(x.split()))*df['sentence_count'].values   
    df['negative_pattern'] = df[target].swifter.apply(get_toxic_index, words = negative_pattern)
    
    negative_verb_pattern = ['smell', 'stink', ' reek', ' hum']
#     df['negative_verb_pattern'] = df[target].swifter.apply(lambda x : sum(x.lower().count(word) for word in negative_verb_pattern)/len(x.split()))*df['sentence_count'].values    
    df['negative_verb_pattern'] = df[target].swifter.apply(get_toxic_index, words = negative_verb_pattern)
    
    ill_pattern = [ 'canser', 'coward', 'disease', 'slave', 'bizarre', 'anorexic', ' cry', 'obese', 'fart','garbage', 'schizophr']
#     df['ill_pattern'] = df[target].swifter.apply(lambda x : sum(x.lower().count(word) for word in ill_pattern)/len(x.split()))*df['sentence_count'].values
    df['ill_pattern'] = df[target].swifter.apply(get_toxic_index, words = ill_pattern)
    
    crime_pattern = ['warmonger', 'pedophile', 'kill', 'murder', 'vandal', 'crime', 'fraud', 'scam', 'gulty', 'speculat']
#     df['crime_pattern'] = df[target].swifter.apply(lambda x : sum(x.lower().count(word) for word in crime_pattern)/len(x.split()))*df['sentence_count'].values
    df['crime_pattern'] = df[target].swifter.apply(get_toxic_index, words = crime_pattern)
    
    nasty_pattern = ['nasty', 'bad ', 'black', 'filth', 'horrible','horror','suicid', 'masturb', 'disgust', 'slime', 'slur']
#     df['nasty_pattern'] = df[target].swifter.apply(lambda x : sum(x.lower().count(word) for word in nasty_pattern)/len(x.split()))*df['sentence_count'].values
    df['nasty_pattern'] = df[target].swifter.apply(get_toxic_index, words = nasty_pattern)
    
    talkative_pattern = ['talker', 'chatterbox', 'windbag', 'babbler', 'gossip', 'jay'] 
#     df['talkative_pattern'] = df[target].swifter.apply(lambda x : sum(x.lower().count(word) for word in talkative_pattern)/len(x.split()))*df['sentence_count'].values
    df['talkative_pattern'] = df[target].swifter.apply(get_toxic_index, words = talkative_pattern)
    
    target_pattern = ['self', 'selves'] 
#     df['target_pattern'] = df[target].swifter.apply(lambda x : sum(x.lower().count(word) for word in target_pattern)/len(x.split()))*df['sentence_count'].values
    df['target_pattern'] = df[target].swifter.apply(get_toxic_index, words = target_pattern)
    
    WTF_pattern = ['what the', 'wtf', 'omfg', 'why the'] 
#     df['WTF_pattern'] = df[target].swifter.apply(lambda x : sum(x.lower().count(word) for word in WTF_pattern)/len(x.split()))*df['sentence_count'].values
    df['WTF_pattern'] = df[target].swifter.apply(get_toxic_index, words = WTF_pattern)
    
    country_pattern = ['moldavian', 'iraq', 'egypt','romanian','countr', 'israel', 'russia', 'europa', 'america', 'livin',  'arab','chinese','czech',
                       'china', 'spain', 'germanic', 'germany', 'poland','texas', 'russkie', 'france', 'armenia', 'rebublic', 'macedonia', 'bulgar',
                       'albania',' turk', 'london','greek', 'natioa', 'population', 'india', 'hindu', 'turk', 'saudi', 'pakistan'] 
#     df['country_pattern'] = df[target].swifter.apply(lambda x : sum(x.lower().count(word) for word in country_pattern)/len(x.split()))*df['sentence_count'].values
    df['country_pattern'] = df[target].swifter.apply(get_toxic_index, words = country_pattern)
    
    politics_pattern = ['putin', 'bush', 'abama', 'trump', 'stalin', 'kgb', 'saddam', 'hussein', 'politic', 'president', 'cult',
                        'anarchist', 'osama bin', 'hitler', ' war', ' fbi', 'lenin', 'biden', 'liberal', 'propagand']  
#     df['politics_pattern'] = df[target].swifter.apply(lambda x : sum(x.lower().count(word) for word in politics_pattern)/len(x.split()))*df['sentence_count'].values
    df['politics_pattern'] = df[target].swifter.apply(get_toxic_index, words = politics_pattern)
    
    NO_pattern = [' no', 'nooo', 'not','nor', 'none', 'no one', 'nobody', 'nothing', 'neither', 'nowhere', 'never', 'negative', ' un-', 'anti', 'pseudo'] 
#     df['NO_pattern'] = df[target].swifter.apply(lambda x : sum(x.lower().count(word) for word in NO_pattern)/len(x.split()))*df['sentence_count'].values
    df['NO_pattern'] = df[target].swifter.apply(get_toxic_index, words = NO_pattern)
    
    NO_verb_pattern = ["do't", "do ' t", 'doesn’t', 'isn’t', 'wasn’t', 'shouldn’t', 'wouldn’t', "couldn’t", "can't", "can ' t", "cant"] 
#     df['NO_verb_pattern'] = df[target].swifter.apply(lambda x : sum(x.lower().count(word) for word in NO_verb_pattern)/len(x.split()))*df['sentence_count'].values
    df['NO_verb_pattern'] = df[target].swifter.apply(get_toxic_index, words = NO_verb_pattern)
    
    max_pattern = ["all!", 'every', 'any', 'too ', 'such', 'very'] 
#     df['max_pattern'] = df[target].swifter.apply(lambda x : sum(x.lower().count(word) for word in max_pattern)/len(x.split()))*df['sentence_count'].values
    df['max_pattern'] = df[target].swifter.apply(get_toxic_index, words = max_pattern)
    
    thanks_pattern = ["sorry", " thank", 'thx', 'hello', 'hey']  
#     df['thanks_pattern'] = df[target].swifter.apply(lambda x : sum(x.lower().count(word) for word in thanks_pattern)/len(x.split()))*df['sentence_count'].values
    df['thanks_pattern'] = df[target].swifter.apply(get_toxic_index, words = thanks_pattern)
    
    positive_pattern = ["well", "nice", 'good', 'cool'] 
#     df['positive_pattern'] = df[target].swifter.apply(lambda x : sum(x.lower().count(word) for word in positive_pattern)/len(x.split()))*df['sentence_count'].values
    df['positive_pattern'] = df[target].swifter.apply(get_toxic_index, words = positive_pattern)
    
    weapon_pattern = ["gun", "weapon", 'rocket', 'shoot', 'bomb'] 
#     df['weapon_pattern'] = df[target].swifter.apply(lambda x : sum(x.lower().count(word) for word in weapon_pattern)/len(x.split()))*df['sentence_count'].values
    df['weapon_pattern'] = df[target].swifter.apply(get_toxic_index, words = weapon_pattern)

    new_features = [i for i in df.columns if i not in col_before]
    
    if show_fs: print('new_features', len(new_features), new_features)
    
    return df, new_features


# Датасеты

In [ ]:
df_val = pd.read_csv("../input/jigsaw-toxic-severity-rating/validation_data.csv")

gp1=df_val.copy()

# Create a hash function for every unique pair
gp1['pair'] = gp1.swifter.apply(lambda x:" ".join(sorted((x['less_toxic'],
                                                  x['more_toxic']))),axis=1)
gp1['pair_hash'] = gp1.pair.swifter.apply(lambda x: str(abs(hash(x)) % (10 ** 8)))
del gp1['pair']

print(f'No. of rows in val_data: {len(gp1)} and no. of unique sentence pairs: {len(gp1.pair_hash.drop_duplicates())}')

############
gp1['pair_cnt']=gp1.groupby(['pair_hash'])['worker'].transform(lambda x: x.count())

gp1['cnt']=gp1.groupby(['pair_hash', 
                        'less_toxic',
                        'more_toxic'])['worker'].transform(lambda x: x.count())

print(gp1[['less_toxic','more_toxic','cnt']].drop_duplicates().cnt.value_counts())

# # Cases with 3 unique rating and 1 disagreement 

gp1[(gp1.pair_cnt == 3) & (gp1.cnt == 1)].sample(10)

df_val2 = gp1[~((gp1.pair_cnt == 3) & (gp1.cnt == 1))][['worker', 'less_toxic', 'more_toxic']]
print(df_val2.shape)

df_val3 = df_val2[['less_toxic', 'more_toxic']].drop_duplicates()
print(df_val3.shape) 

cleaning = True

if cleaning:
    df_val_l, _ = make_features(df_val[['worker','less_toxic']].rename(columns={'less_toxic': 'text'}), target = 'text')
    df_val_l.drop('worker', axis=1, inplace=True)
    df_val_l = clean(df_val_l, 'text') 
    print('теперь для more toxic')
    df_val_m, _ = make_features(df_val[['worker','more_toxic']].rename(columns={'more_toxic': 'text'}), target = 'text')
    df_val_m.drop('worker', axis=1, inplace=True)
    df_val_m = clean(df_val_m, 'text') 
    
if cleaning:
    df_val2_l, _ = make_features(df_val2[['worker','less_toxic']].rename(columns={'less_toxic': 'text'}), target = 'text')
    df_val2_l.drop('worker', axis=1, inplace=True)
    df_val2_l = clean(df_val2_l, 'text') 
    
    df_val2_m, _ = make_features(df_val2[['worker','more_toxic']].rename(columns={'more_toxic': 'text'}), target = 'text')
    df_val2_m.drop('worker', axis=1, inplace=True)
    df_val2_m = clean(df_val2_m, 'text')        
    
if cleaning:
    df_val3['worker'] = 1 # Костыль
    df_val3_l, _ = make_features(df_val3[['worker','less_toxic']].rename(columns={'less_toxic': 'text'}), target = 'text')
    df_val3_l.drop('worker', axis=1, inplace=True)
    df_val3_l = clean(df_val3_l, 'text') 
    
    df_val3_m, _ = make_features(df_val3[['worker','more_toxic']].rename(columns={'more_toxic': 'text'}), target = 'text')
    df_val3_m.drop('worker', axis=1, inplace=True)
    df_val3_m = clean(df_val3_m, 'text')   


# ✅ Model 1 Ridge + Lasso + toxic-comment-classification dataset

In [ ]:
df = pd.read_csv("../input/jigsaw-toxic-comment-classification-challenge/train.csv")
print(df.shape) 
df.sample(4)


df['severe_toxic'] = df.severe_toxic * 3
df['threat'] = df.threat * 2

df['y'] = (df[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].sum(axis=1) ).astype(int)
df['y'] = np.around (df["y"].values , decimals = 4)

df.rename(columns={'comment_text': 'text'}, inplace=True)
df = df[['text','y']]
# Reduce rows with 0 toxicity
df = pd.concat([df[df.y>0] , 
                df[df.y==0].sample(int(len(df[df.y>0])*1.5)) ], axis=0).sample(frac=1)

print(df.shape)

df.columns

In [ ]:
# TODO расстояние левинштейна

df, handle_features = make_features(df.copy(), target = 'text', show_fs = True)

handle_features += ['URL_count']
print(handle_features)
# cor = df.corr()
df.sample(5)

In [ ]:
df.info()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt


cor = df[df['toxic_word_pattern'] >= 0.1].corr()

f, ax = plt.subplots(figsize=(25,25))
sns.heatmap(cor, vmin = 0.05, square=True)

In [ ]:
df.upper_on_lower_word.hist()

In [ ]:
text_features = ['text']
handle_features

# df = df[text_features + handle_features + ['y']]
# df

In [ ]:
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.feature_selection import SelectKBest, f_regression, f_classif

In [ ]:
ridge_txt_pipeline = Pipeline(steps=[
    ('col_selector', ColumnSelector(cols=('text'),drop_axis=True)),
    ("tfidf", TfidfVectorizer(
        min_df = 5, max_df = 0.7, 
        norm = 'l2', analyzer = 'char_wb', ngram_range = (3, 5))),
    ("Ridge", Ridge(alpha=1, max_iter = 600, random_state=42)), 
])


lasso_transformer = Pipeline(steps=[
    ('col_selector', ColumnSelector(cols=handle_features, drop_axis=True)),
    ("imputer", StandardScaler()),
    ("fs", SelectKBest(score_func=f_regression, k=60)),
    ("Ridge", Lasso(random_state=42)), 
])

ridge_fs_transformer = Pipeline(steps=[
    ('col_selector', ColumnSelector(cols=handle_features, drop_axis=True)),
    ("imputer", StandardScaler()),
#     ("fs", SelectKBest(score_func=f_regression, k=60)),
    ("Ridge", Ridge(alpha=1, max_iter = 700, random_state=42)), 
])

#   Для численных/бертовых - применяем SimpleImputer,
rf_num_pipeline = Pipeline(steps=[
#     ("imputer", SimpleImputer()),
    ('col_selector', ColumnSelector(cols=handle_features,drop_axis=True)),
    ("imputer", StandardScaler()),
#     ("fs", SelectKBest(score_func=f_regression, k=50)),
    ("RF", RandomForestRegressor(max_depth=9, n_estimators=100, min_samples_leaf = 7,
                                 n_jobs=-1, verbose=True, random_state = 42)),     
])

fs_transformer = Pipeline(steps=[
    ('col_selector', ColumnSelector(cols=handle_features, drop_axis=True)),
    ("imputer", StandardScaler()),
])

text_transformer = Pipeline(steps=[
    ('col_selector', ColumnSelector(cols=('text'),drop_axis=True)),
    ("tfidf", TfidfVectorizer( min_df = 5, max_df = 0.7,  norm = 'l2', analyzer = 'char_wb', ngram_range = (3, 5))),
])

# Собираем воедино трансформеры для текстовых и бертовых признаков
big_ridge_transformer = ColumnTransformer(transformers=[
    ("text", text_transformer, text_features),
    ("hand_fs", fs_transformer, handle_features),
#     ("Ridge", Ridge(alpha=1, max_iter = 2000, random_state=42)), 
])
big_ridge_pipline = Pipeline(steps=[
    ("col_selector", big_ridge_transformer),
    ("Ridge", Ridge(alpha=1, max_iter = 600, random_state=42)),
])


pipeline = VotingRegressor(n_jobs = -1, 
    estimators=[
#                 ("LinReg",  Ridge(alpha=1/2 )), #LinearRegression(normalize=True)), 
#         ('lasso', Lasso(random_state=42)),
        ('RandForest_fs', rf_num_pipeline),
        ('Lasso_fs', lasso_transformer),
#         ("Ridge", Ridge(alpha=1, max_iter = 2000, random_state=42)), 
        ("Ridge_fs", ridge_fs_transformer), 
        ("Big_Ridge", big_ridge_pipline), 
         ("Ridge_txt", ridge_txt_pipeline), 
               ] 
)

pipeline

In [ ]:
cleaning = True

if cleaning:
    df = clean(df.copy(), 'text')

pipeline.fit(df.drop('y', axis=1), df['y'])
# pipeline.predict(df.drop('y', axis=1))

In [ ]:

# parameters = {
# #     "tfidf__max_df": [1],#  0.75, 0.9, 1.0),
# #     'vect__max_features': (None, 5000, 10000, 50000),
# #     "tfidf__ngram_range": [(2, 5)], # (1, 2), (2, 3), (2, 4), (2, 5)),  # unigrams or bigrams
# #     'tfidf__use_idf': (True, False),
# #     'tfidf__stop_words' : True, 
# #     'tfidf__norm': ('l1', 'l2'),
#     "RF__n_estimators": [100],
#     'fs__k': [55, 'all'],
#     "RF__max_depth": [9],
#     'RF__min_samples_leaf' : [5, 6],
# #     'tfidf__min_df' : [5], # 3, 5, 7),
# #     "clf__alpha": [1], # 0.3, 0.5, 0.7, 0.9, 1.0),
# #     "clf__penalty": ("l2", "elasticnet"),
# #     'cv' : 3, 
# #     'clf__max_iter': (10, 50, 80),
# }
# gr_pipline = num_transformer

# grid_search = GridSearchCV(gr_pipline, parameters, 
#                            cv = 3,
#                            n_jobs=-1, verbose=1)

# print("Performing grid search...")
# print("pipeline:", [name for name, _ in gr_pipline.steps])
# print("parameters:")
# pprint(parameters)
# t0 = time()
# grid_search.fit(df.drop('y', axis=1), df['y'])
# print("done in %0.3fs" % (time() - t0))

# print("Best score: %0.3f" % grid_search.best_score_)
# print("Best parameters set:")
# best_parameters = grid_search.best_estimator_.get_params()
# for param_name in sorted(parameters.keys()):
#     print("\t%s: %r" % (param_name, best_parameters[param_name]))

In [ ]:
handle_features

In [ ]:
def plot_importance(handle_features, pipeline, height, top_n=50, square=False, model='rf'):
    
    fi = pd.DataFrame(index = handle_features, columns = [])
#     for i, m in enumerate(best_model):
#         fi[f'm_{i}'] = pipeline['Ridge'].coef_
    if model == 'rf':
        fi[f'm_{0}'] = pipeline['RF'].feature_importances_
    elif model == 'ridge':
        if square:
            fi[f'm_{0}'] = abs(pipeline['Ridge'].coef_)
        else:
            fi[f'm_{0}'] = pipeline['Ridge'].coef_

    fi = fi.stack().reset_index().iloc[:,[0, 2]]#.to_frame()
    fi.columns = ['feature','importance']

    cols_ord = list(fi.groupby('feature')['importance'].mean().sort_values(ascending=False).index)
    print('Всего признаков', len(cols_ord), 'Усреднее по {}-ти моделям: '.format(1) )
    cols_ord = cols_ord[:top_n]
    
    fi = fi[fi['feature'].isin(cols_ord)]
    
    plt.figure(figsize=(10, len(cols_ord)*height))
    b = sns.boxplot(data=fi, y='feature', x='importance', orient='h', order=cols_ord)
    
    print('На график нанесено топ-{} признаков'.format(top_n))
    return fi.groupby(by =['feature'], as_index=False)['importance'].mean().sort_values(by='importance', ascending=False)


# rf_num_pipeline.fit(df.drop('y', axis=1), df['y'])
# df_feats_imp = plot_importance(handle_features, rf_num_pipeline, 0.20, top_n=70, square = 0)
# num_transformer['RF'].feature_importances_

# pipeline['Ridge'].coef_

# Validate on actual val data 📈

# Predict on test data

In [ ]:
# Validate on modified val data  
    
p1 = pipeline.predict(df_val_l)
p2 = pipeline.predict(df_val_m)
print('MSE',sum((p1-p2)**2)/len(p1)) 
print('good MSE',sum(((p1-p2)*(p1<p2))**2)/len(p1)) 
print('bad MSE',sum(((p1-p2)*(p1>p2))**2)/len(p1)) 
print(f'Validation Accuracy from Model 1 is { np.round((p1 < p2).mean() * 100, 2)}')

# Validate on modified val2 data    
    
p1 = pipeline.predict(df_val2_l)
p2 = pipeline.predict(df_val2_m)
print('MSE',sum((p1-p2)**2)/len(p1)) 
print('good MSE',sum(((p1-p2)*(p1<p2))**2)/len(p1)) 
print('bad MSE',sum(((p1-p2)*(p1>p2))**2)/len(p1)) 
print(f'Validation Accuracy from Model 1 is { np.round((p1 < p2).mean() * 100, 2)}')
# Validate on modified val3 data
    
    
p1 = pipeline.predict(df_val3_l)
p2 = pipeline.predict(df_val3_m)

print('MSE',sum((p1-p2)**2)/len(p1)) 
print('good MSE',sum(((p1-p2)*(p1<p2))**2)/len(p1)) 
print('bad MSE',sum(((p1-p2)*(p1>p2))**2)/len(p1)) 
      
print(f'Validation Accuracy from Model 1 is { np.round((p1 < p2).mean() * 100, 2)}')
 

In [ ]:
# MSE 1.2850912490689608
# MSE 1.1155849628182188
# Validation Accuracy from Model 1 is 68.42
# MSE 1.405483622204499
# MSE 1.3218968138615883
# Validation Accuracy from Model 1 is 75.12
# MSE 1.2879478406265237
# MSE 1.1925015694734544
# MSE 0.09544627115307117
# Validation Accuracy from Model 1 is 72.93

# Анализ ошибок 🤔

In [ ]:
df_val3_l['less_toxic'] = df_val3_l['text']
df_val3_l['more_toxic'] = df_val3_m['text']
df_val3_l['less_score'] = p1
df_val3_l['more_score'] = p2
df_val3_l['delta_correct'] = p1 - p2
df_val3_l['correct'] = p1 < p2

# df_val3_l['less_len'] = df_val3_l['less_toxic'].swifter.apply(len)
# df_val3_l['more_len'] = df_val3_l['more_toxic'].swifter.apply(len)
# df_val3_l['lenner'] = df_val3_l['less_len'].values - df_val3_l['more_len'].values


pd.options.display.max_colwidth = 800
pd.options.display.max_rows = 80

df_val3_l[['less_toxic','more_toxic', 'correct','less_score', 'more_score','delta_correct']].sort_values('delta_correct', ascending=False).head(2000).sample(50)     

In [ ]:
# plt.scatter(df_val3_l.lenner, df_val3_l.delta_correct)
# sns.scatterplot(df_val3_l, y = 'lenner', x = "delta_correct")
# df_val3_l_tmp = df_val3_l
# df_val3_l_tmp['lenner'] = abs(df_val3_l.lenner.values)
# df_val3_l_tmp['delta_correct'] = abs(df_val3_l.delta_correct.values)
# df_val3_l_tmp.plot.scatter(x= 'lenner', y ="delta_correct")


# TODO убрать скоррелированные фичи!!!

In [ ]:
df_test = pd.read_csv("../input/jigsaw-toxic-severity-rating/comments_to_score.csv")

if cleaning:
    
    df_test, _ = make_features(df_test, target = 'text')
    df_test = clean(df_test, 'text')
    coms_id = df_test.comment_id.values
    df_test.drop('comment_id', axis=1, inplace=True)

m1_preds = pipeline.predict(df_test)

df_test['score_1'] = rankdata(m1_preds, method='ordinal') # m1_preds
df_test['comment_id'] = coms_id


with open("pipeline.pkl", "wb") as f:
    pickle.dump(pipeline, f)


# ✅ Model 2 - Ridge + regression-based data V2

In [ ]:
df = pd.read_csv("../input/jigsaw-regression-based-data/train_data_version2.csv")[['text', 'y']]

if cleaning:
    
#     df, _ = make_features(df[['text','y']], target = 'text')
    df = clean(df, 'text')
    
print(df.shape)

y=np.around(df["y"].values, decimals = 2)

In [ ]:
ridge_pipeline_1 = Pipeline(steps=[
    ('col_selector', ColumnSelector(cols=('text'), drop_axis=True)),
    ("tfidf", TfidfVectorizer(min_df = 1, max_df = 0.7, analyzer = 'char_wb', ngram_range = (2, 5))),
    ("Ridge", Ridge(alpha=1, max_iter = 600, random_state=42)), 
])

# # #   Для численных/бертовых - применяем SimpleImputer,
# # num_transformer = Pipeline(steps=[
# #     ('col_selector', ColumnSelector(cols=handle_features,drop_axis=True)),
# #     ("imputer", StandardScaler()),
# #     ("fs", SelectKBest(score_func=f_regression, k=50)),
# #     ("RF", RandomForestRegressor(max_depth=8, n_estimators=100, min_samples_leaf = 7,
# #                                  n_jobs=-1, verbose=True, random_state = 42)), 
# # ])

# # # Собираем воедино трансформеры для текстовых и ручных признаков
# # data_transformer = ColumnTransformer(transformers=[
# #     ("text_fs", ridge_pipline, text_features),
# #     ("bert_fs", num_transformer, handle_features),
# # ])

# # big_ridge_pipline = Pipeline(steps=[
# #     ('col_selector', ColumnSelector(cols=handle_features,drop_axis=True)),
# #     ("imputer", StandardScaler()),
# #     ("fs", SelectKBest(score_func=f_regression, k=50)),
# #     ("RF", RandomForestRegressor(max_depth=8, n_estimators=100, min_samples_leaf = 7,
# #                                  n_jobs=-1, verbose=True, random_state = 42)), 
# # ])

    
# # ridge_pipeline_1 = VotingRegressor(
# #     estimators=[
# # #                 ("LinReg",  Ridge(alpha=1/2 )), #LinearRegression(normalize=True)), 
# # #         ('lasso', Lasso(random_state=42)),
# #         ('lasso', num_transformer),
# # #         ("Ridge", Ridge(alpha=1, max_iter = 600, random_state=42)), 
# #         ("Ridge", ridge_pipline), 
# #                ] 
# # )
# # ridge_pipeline_1


# ridge_txt_pipeline = Pipeline(steps=[
#     ('col_selector', ColumnSelector(cols=('text'),drop_axis=True)),
#     ("tfidf", TfidfVectorizer(analyzer='char_wb', max_df=0.95, min_df=13, ngram_range=(2, 5))),
#     ("Ridge", Ridge(alpha=1, random_state=42)), 
# ])


# lasso_transformer = Pipeline(steps=[
#     ('col_selector', ColumnSelector(cols=handle_features, drop_axis=True)),
#     ("imputer", StandardScaler()),
# #     ("fs", SelectKBest(score_func=f_regression, k=60)),
#     ("Ridge", Lasso(random_state=42)), 
# ])

# ridge_fs_transformer = Pipeline(steps=[
#     ('col_selector', ColumnSelector(cols=handle_features, drop_axis=True)),
#     ("imputer", StandardScaler()),
# #     ("fs", SelectKBest(score_func=f_regression, k=60)),
#     ("Ridge", Ridge(alpha=1, max_iter = 400, random_state=42)), 
# ])

# #   Для численных/бертовых - применяем SimpleImputer,
# rf_num_pipeline = Pipeline(steps=[
# #     ("imputer", SimpleImputer()),
#     ('col_selector', ColumnSelector(cols=handle_features,drop_axis=True)),
#     ("imputer", StandardScaler()),
# #     ("fs", SelectKBest(score_func=f_regression, k=50)),
#     ("RF", RandomForestRegressor(max_depth=8, n_estimators=100, min_samples_leaf = 7,
#                                  n_jobs=-1, verbose=True, random_state = 42)),     
# ])

# fs_transformer = Pipeline(steps=[
#     ('col_selector', ColumnSelector(cols=handle_features, drop_axis=True)),
#     ("imputer", StandardScaler()),
# ])

# text_transformer = Pipeline(steps=[
#     ('col_selector', ColumnSelector(cols=('text'),drop_axis=True)),
#     ("tfidf", TfidfVectorizer(analyzer='char_wb', max_df=0.7, min_df=1, ngram_range=(2, 5))),
# ])

# # Собираем воедино трансформеры для текстовых и бертовых признаков
# big_ridge_transformer = ColumnTransformer(transformers=[
#     ("text", text_transformer, text_features),
#     ("hand_fs", fs_transformer, handle_features),
# #     ("Ridge", Ridge(alpha=1, max_iter = 500, random_state=42)), 
# ])
# big_ridge_pipline = Pipeline(steps=[
#     ("col_selector", big_ridge_transformer),
#     ("Ridge", Ridge(alpha=1, max_iter = 500, random_state=42)),
# ])

    
# ridge_pipeline_1 = VotingRegressor(n_jobs = -1, 
#     estimators=[
#         ('RandForest_fs', rf_num_pipeline),
#         ('Lasso_fs', lasso_transformer),
#         ("Ridge_fs", ridge_fs_transformer), 
#         ("Big_Ridge", big_ridge_pipline), 
#         ("Ridge_txt", ridge_txt_pipeline), 
#                ] 
# )


ridge_pipeline_1.fit(df.drop('y', axis=1), y)

In [ ]:
# ridge_txt_pipeline

In [ ]:
# Validation Accuracy from Model 1 is 67.31 # Best Ridge only
# Validation Accuracy from Model 1 is 73.49
# Validation Accuracy from Model 1 is 71.38


  
p1 = ridge_pipeline_1.predict(df_val_l)
p2 = ridge_pipeline_1.predict(df_val_m)

print(f'Validation Accuracy from Model 1 is { np.round((p1 < p2).mean() * 100, 2)}')
        
p1 = ridge_pipeline_1.predict(df_val2_l)
p2 = ridge_pipeline_1.predict(df_val2_m)

print(f'Validation Accuracy from Model 1 is { np.round((p1 < p2).mean() * 100, 2)}')
    
p1 = ridge_pipeline_1.predict(df_val3_l)
p2 = ridge_pipeline_1.predict(df_val3_m)

print(f'Validation Accuracy from Model 1 is { np.round((p1 < p2).mean() * 100, 2)}')
 
    
# Validation Accuracy from Model 1 is 65.93 onli ridge
# Validation Accuracy from Model 1 is 71.52
# Validation Accuracy from Model 1 is 69.54


# Validation Accuracy from Model 1 is 67.39
# Validation Accuracy from Model 1 is 73.61
# Validation Accuracy from Model 1 is 71.5

In [ ]:
jr_preds=ridge_pipeline_1.predict(df_test)
df_test['score_2']=rankdata(jr_preds, method='ordinal') 

with open("ridge_pipeline_1.pkl", "wb") as f:
    pickle.dump(ridge_pipeline_1, f)


# ✅ Model 3 - Ridge + Ruddit dataset

In [ ]:
df = pd.read_csv("../input/d/rajkumarl/ruddit-jigsaw-dataset/Dataset/ruddit_with_text.csv")[['txt', 'offensiveness_score']].rename(columns={'txt': 'text', 'offensiveness_score' : 'y'})[['text', 'y']]

if cleaning:
#     df, _ = make_features(df[['text','y']], target = 'text')
    df = clean(df, 'text')

print(f"rud_df:{df.shape}")

y = np.around (df["y"].values, decimals = 1)

In [ ]:

ridge_pipeline_2 = Pipeline(steps=[
    ('col_selector', ColumnSelector(cols=('text'),drop_axis=True)),
    ("tfidf", TfidfVectorizer(analyzer='char_wb', max_df=0.7, min_df=3, ngram_range=(3, 4))),
    ("Ridge", Ridge(alpha=1, max_iter = 600, random_state=42)), 
])


# lasso_transformer = Pipeline(steps=[
#     ('col_selector', ColumnSelector(cols=handle_features, drop_axis=True)),
#     ("imputer", StandardScaler()),
# #     ("fs", SelectKBest(score_func=f_regression, k=60)),
#     ("Ridge", Lasso(random_state=42)), 
# ])

# ridge_fs_transformer = Pipeline(steps=[
#     ('col_selector', ColumnSelector(cols=handle_features, drop_axis=True)),
#     ("imputer", StandardScaler()),
# #     ("fs", SelectKBest(score_func=f_regression, k=60)),
#     ("Ridge", Ridge(alpha=1, max_iter = 600, random_state=42)), 
# ])

# #   Для численных/бертовых - применяем SimpleImputer,
# rf_num_pipeline = Pipeline(steps=[
# #     ("imputer", SimpleImputer()),
#     ('col_selector', ColumnSelector(cols=handle_features,drop_axis=True)),
#     ("imputer", StandardScaler()),
# #     ("fs", SelectKBest(score_func=f_regression, k=50)),
#     ("RF", RandomForestRegressor(max_depth=9, n_estimators=80, min_samples_leaf = 3,
#                                  n_jobs=-1, verbose=True, random_state = 42)),     
# ])

# fs_transformer = Pipeline(steps=[
#     ('col_selector', ColumnSelector(cols=handle_features, drop_axis=True)),
#     ("imputer", StandardScaler()),
# ])

# text_transformer = Pipeline(steps=[
#     ('col_selector', ColumnSelector(cols=('text'),drop_axis=True)),
#     ("tfidf", TfidfVectorizer(analyzer='char_wb', max_df=0.7, min_df=3, ngram_range=(3, 4))),
# ])

# # Собираем воедино трансформеры для текстовых и бертовых признаков
# big_ridge_transformer = ColumnTransformer(transformers=[
#     ("text", text_transformer, text_features),
#     ("hand_fs", fs_transformer, handle_features),
# #     ("Ridge", Ridge(alpha=1, max_iter = 600, random_state=42)), 
# ])
# big_ridge_pipline = Pipeline(steps=[
#     ("col_selector", big_ridge_transformer),
#     ("Ridge", Ridge(alpha=1, max_iter = 600, random_state=42)),
# ])

    
# ridge_pipeline_2 = VotingRegressor(n_jobs = -1, 
#     estimators=[
#         ('RandForest_fs', rf_num_pipeline),
#         ('Lasso_fs', lasso_transformer),
#         ("Ridge_fs", ridge_fs_transformer), 
#         ("Big_Ridge", big_ridge_pipline), 
#          ("Ridge_txt", ridge_txt_pipeline), 
#                ] 
# )


ridge_pipeline_2.fit(df.drop('y', axis=1), y)

In [ ]:

  
p1 = ridge_pipeline_2.predict(df_val_l)
p2 = ridge_pipeline_2.predict(df_val_m)

print(f'Validation Accuracy from Model 1 is { np.round((p1 < p2).mean() * 100, 2)}')
        
p1 = ridge_pipeline_2.predict(df_val2_l)
p2 = ridge_pipeline_2.predict(df_val2_m)

print(f'Validation Accuracy from Model 1 is { np.round((p1 < p2).mean() * 100, 2)}')
    
p1 = ridge_pipeline_2.predict(df_val3_l)
p2 = ridge_pipeline_2.predict(df_val3_m)

print(f'Validation Accuracy from Model 1 is { np.round((p1 < p2).mean() * 100, 2)}')

# # Validation Accuracy from Model 1 is 63.01
# # Validation Accuracy from Model 1 is 67.48
# # Validation Accuracy from Model 1 is 65.78


# # Validation Accuracy from Model 1 is 62.94
# # Validation Accuracy from Model 1 is 67.41
# # Validation Accuracy from Model 1 is 65.73

In [ ]:
rud_preds=ridge_pipeline_2.predict(df_test)
df_test['score_3']=rankdata(rud_preds, method='ordinal') 

with open("ridge_pipeline_2.pkl", "wb") as f:
    pickle.dump(ridge_pipeline_2, f)

# Make Submission

In [ ]:
df_test['score']= df_test['score_1']*0.2 + df_test['score_2']*0.4 + df_test['score_3']*0.3
df_test['score'] =rankdata( df_test['score'].values, method='ordinal') 
df_test[['comment_id','score']].to_csv('submission.csv', index=False)